# 01 - Ejecutar Provincia Individual

Este notebook ejecuta el pipeline completo para **una sola provincia** con intervalo de años personalizado.

## Características

- Selección interactiva de provincia
- Intervalo de años configurable (inicio y fin)
- Optimizaciones automáticas (multi-core)
- Modo público activado (no exporta data sensible)

## Instrucciones

1. Instalar dependencias: `pip install -r requirements.txt`
2. Colocar raw en `data/raw/` o configurar en `configs/provincias.yaml`
3. Ejecutar celda y seguir prompts interactivos

## Nota

Para procesar **todas las provincias**, usar `00_run_pais.ipynb`

In [ ]:
from pathlib import Path
import os
import sys
import shutil
import importlib
import subprocess
import yaml
from time import perf_counter

# Navegar al root del proyecto
root = Path.cwd().resolve()
while root != root.parent and not (root / "configs").exists():
    root = root.parent
os.chdir(root)

if str(root) not in sys.path:
    sys.path.insert(0, str(root))

# Optimizaciones de rendimiento
print("="*60)
print("INICIALIZANDO")
print("="*60)

try:
    from src.utils.performance import enable_pandas_performance, get_optimal_workers
    enable_pandas_performance()
    workers = get_optimal_workers()
    print(f"✓ Optimizaciones habilitadas")
    print(f"✓ CPU cores: {os.cpu_count()}")
    print(f"✓ Workers: {workers}")
except Exception as e:
    print(f"⚠ Optimizaciones no disponibles: {e}")
    workers = 1

# Verificar geopandas
try:
    import geopandas as gpd
    print(f"✓ geopandas {gpd.__version__}")
except Exception:
    print("Instalando geopandas...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "geopandas"])
    import geopandas as gpd
    print(f"✓ geopandas {gpd.__version__}")

# Reload modules
import src.reporting.export_artifacts as export_artifacts
importlib.reload(export_artifacts)
from src.reporting.export_artifacts import run_provincia

# Cargar provincias disponibles
provincias_cfg_path = Path("configs") / "provincias.yaml"
provincias_cfg = {}
if provincias_cfg_path.exists():
    provincias_cfg = yaml.safe_load(provincias_cfg_path.read_text(encoding="utf-8")) or {}

prov_map = provincias_cfg.get("provincias", {}) or {}

# Ordenar con PROVINCIA al final
all_provs = sorted(prov_map.keys())
if "PROVINCIA" in all_provs:
    all_provs.remove("PROVINCIA")
    available_provinces = all_provs + ["PROVINCIA"]
else:
    available_provinces = all_provs

print("\n" + "="*60)
print("SELECCIÓN DE PROVINCIA")
print("="*60)
print("\nProvincias disponibles:")
for idx, prov in enumerate(available_provinces, start=1):
    print(f"  {idx:02d}. {prov}")

# Selección de provincia
selected_province = None
while selected_province is None:
    choice = input("\nProvincia (número o nombre): ").strip()
    if not choice:
        print("⚠ Debes ingresar una provincia válida.")
        continue
    if choice.isdigit():
        idx = int(choice)
        if 1 <= idx <= len(available_provinces):
            selected_province = available_provinces[idx - 1]
            break
    else:
        choice_norm = choice.upper().strip()
        if choice_norm in available_provinces:
            selected_province = choice_norm
            break
    print("⚠ Provincia inválida, intenta otra vez.")

print(f"\n✓ Provincia seleccionada: {selected_province}")

# Selección de intervalo
print("\n" + "="*60)
print("INTERVALO DE AÑOS")
print("="*60)

use_all_years = input("\n¿Usar todo el intervalo 2000-2024? (s/N): ").strip().lower() == "s"

if use_all_years:
    start_year = 2000
    end_year = 2024
else:
    # Pedir año inicio
    while True:
        start_input = input("Año inicio (2000-2024): ").strip()
        try:
            start_year = int(start_input)
            if 2000 <= start_year <= 2024:
                break
            print("⚠ Año debe estar entre 2000 y 2024")
        except ValueError:
            print("⚠ Ingresa un año válido")
    
    # Pedir año fin
    while True:
        end_input = input(f"Año fin ({start_year}-2024): ").strip()
        try:
            end_year = int(end_input)
            if start_year <= end_year <= 2024:
                break
            print(f"⚠ Año debe estar entre {start_year} y 2024")
        except ValueError:
            print("⚠ Ingresa un año válido")

print(f"\n✓ Intervalo: {start_year}-{end_year}")

# Modo público (siempre activado)
public_mode = True
print(f"✓ Modo público: activado (datos anonimizados)")

# Construir config override con intervalo personalizado
def build_configs_override(start_year: int, end_year: int, use_all_years: bool) -> Path:
    base = Path("configs")
    target = Path("notebooks") / "configs_override"
    target.mkdir(parents=True, exist_ok=True)
    
    global_cfg = yaml.safe_load((base / "global.yaml").read_text(encoding="utf-8")) or {}
    if not use_all_years:
        global_cfg["window_start_year"] = int(start_year)
        global_cfg["window_end_year"] = int(end_year)
    (target / "global.yaml").write_text(yaml.safe_dump(global_cfg, sort_keys=False), encoding="utf-8")
    
    if (base / "provincias.yaml").exists():
        (target / "provincias.yaml").write_text(
            (base / "provincias.yaml").read_text(encoding="utf-8"),
            encoding="utf-8",
        )
    return target

def resolve_raw_path(province: str) -> str | None:
    meta = prov_map.get(province.upper())
    if not meta:
        return None
    return meta.get("raw_path")

# Procesamiento
print("\n" + "="*60)
print("PROCESANDO")
print("="*60 + "\n")

cfg_dir = build_configs_override(start_year, end_year, use_all_years)
raw_path = resolve_raw_path(selected_province)

start_time = perf_counter()

try:
    out_base = run_provincia(
        selected_province,
        configs_dir=str(cfg_dir),
        raw_dir="data/raw",
        raw_path=raw_path,
        public_mode=public_mode,
    )
    elapsed = perf_counter() - start_time
    
    print("\n" + "="*60)
    print("COMPLETADO")
    print("="*60)
    print(f"\n✓ Provincia: {selected_province}")
    print(f"✓ Intervalo: {start_year}-{end_year}")
    print(f"✓ Output: {out_base}")
    print(f"✓ Tiempo: {elapsed:.2f}s ({elapsed/60:.2f} min)")
    
except Exception as e:
    elapsed = perf_counter() - start_time
    print("\n" + "="*60)
    print("ERROR")
    print("="*60)
    print(f"\n✗ {selected_province}: {e}")
    print(f"✗ Tiempo: {elapsed:.2f}s")
    
finally:
    # Cleanup
    shutil.rmtree(cfg_dir, ignore_errors=True)
    print(f"\n✓ Cleanup: configs_override eliminado")

print("\nProceso finalizado!")